# Stability of the AMOC

In [ ]:
import os
import sys
import cmocean
import numpy as np
import xarray as xr
import warnings
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
sys.path.append("..")
%matplotlib inline
matplotlib.rc_file('../rc_file')
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

## $M_{ov}$ calculation

The fundamental idea behind the $M_{ov}$ parameter is the effect of a MOC shutdown has on the salinity in the North Atlantic and hence NADW formation.
If the overturning circulation imports freshwater to the Atlantic, a MOC shutdown will increase salinity which counteracts the shutdown.
Vice versa, a freshwater export due to the MOC will lead to freshening of the North Atlantic in case of a shutdown and hence a strengthening of the shut down state.

The Atlantic freshwater budget in steady state is

$$\left[E - P - R \right] = M_{ov} + M_{az} + M_{dif} + M_{BS}$$

with evaporation $E$, precipitation $P$, runoff $R$, as well as freshwater transport terms due to the overturning circulation $M_{ov}$, the azonal gyre circulation $M_{az}$, diffusion $M_{dif}$, and exchange via the Bering Strait $M_{BS}$.


The overturning freswater transport term $M_{ov}$ is given differently in different publications:

_de Vries et al. (2005)_:

$$M_{ov} = - \frac{1}{S_0} \int \bar{v}(z) \left[ \langle S(z) \rangle - S_0 \right] \,\mathrm{d}z$$

where $\bar{v}(z) = \int v(x,z) \,\mathrm{d}x$ is the zonal integral and $\langle S \rangle (z) = \int S(x,z) \,\mathrm{d}x \, \big/ \int \mathrm{d}x$ is the zonal mean, such that

\begin{align}
M_{ov} & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x} - S_0 \right] \,\mathrm{d}z \\
 & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x} \right] \,\mathrm{d}z + \int \int v(x,z) \, \mathrm{d}x \mathrm{d}z  \tag{1}
\end{align}

_Mecking et al. (2017)_:

$$M_{ov} = - \frac{1}{S_0} \int \int v^*(z) \langle S(z) \rangle \, \mathrm{d}x \mathrm{d}z$$

where $v^*(z) = \langle v \rangle (z) - \bar{v} = \int v(x,z) \,\mathrm{d}x \, \big/ \int \mathrm{d}x - \int \int v(x,z) \,\mathrm{d}x \mathrm{d}z \, \big/ \int \int \mathrm{d}x \mathrm{d}z$ such that

\begin{align}
M_{ov} & = - \frac{1}{S_0} \int \int \left[ \frac{\int v(x,z) \,\mathrm{d}x}{ \int \mathrm{d}x} - \frac{\int \int v(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \int \mathrm{d}x \mathrm{d}z} \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}x \mathrm{d}z \\
 & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x - \frac{\int \int v(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \mathrm{d}z} \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  \\
 & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  + \frac{1}{S_0} \int  \left[ \frac{\int \int v(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \mathrm{d}z} \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z \\
  & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  + \frac{1}{S_0} \int \left[ \frac{ \int \int v(x,z) \,\mathrm{d}x \mathrm{d}z  \times   \int S(x,z) \,\mathrm{d}x }{\int \int \mathrm{d}x \mathrm{d}z } \right] \, \mathrm{d}z \\
  & = - \frac{1}{S_0} \int \left[ \int v(x,z) \,\mathrm{d}x \right] \left[ \frac{\int S(x,z) \,\mathrm{d}x}{\int \mathrm{d}x}\right] \, \mathrm{d}z  + \left[ \frac{1}{S_0} \frac{  \int \int S(x,z) \,\mathrm{d}x \mathrm{d}z}{\int \int \mathrm{d}x \mathrm{d}z } \right] \int \int v(x,z) \,\mathrm{d}x \mathrm{d}z  \tag{2}
\end{align}

Equations (2) and (1) are equal if the reference salinity is equal to the section average: $S_0 = \int \int S(x,z) \,\mathrm{d}x \mathrm{d}z \, \big/ \int \int \mathrm{d}x \mathrm{d}z $

In [ ]:
from tqdm import tqdm_notebook
from paths import file_ex_ocn_ctrl, file_ex_ocn_rcp, file_ex_ocn_lpd, path_prace
from filters import lowpass
from timeseries import IterateOutputCESM
from xr_regression import xr_lintrend

In [ ]:
# high resolution coordinates
j_34 = 810               # latitude number at the Cape of Good Hope
i_SA, i_CGH = 555, 1299  # longitude numbers of South America and the Cape of Good Hope

ds = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)[['VVEL', 'SALT', 'DXT', 'DXU', 'z_t', 'dz']].isel({'time':0, 'nlat':j_34, 'nlon':slice(i_SA,i_CGH)})
S0 = (ds.SALT*ds.DXT*ds.dz).sum(dim=['nlon','z_t'])/(ds.DXT.where(ds.SALT>0)*ds.dz).sum(dim=['nlon','z_t'])
v0 = (ds.VVEL*ds.DXU*ds.dz).sum(dim=['nlon','z_t'])/(ds.DXT.where(ds.VVEL>-1000)*ds.dz).sum(dim=['nlon','z_t'])
L = ds.DXT.sum(dim='nlon')
print(f'S_0 = {S0.values:6.2f} g/kg \nv_0 = {v0.values:6.3f} cm/s \nL   = {L.values/1e5:6.0f} km')

In [ ]:
ds.TLONG[ds.TLONG>180] -= 360

X, Y = np.meshgrid(ds.TLONG, -ds.z_t/1e5)
v0_str, S0_str = r'$v_0$', r'$S_0$'

f,ax = plt.subplots(2,3, figsize=(8,7), sharey=True)
for i in range(2):
    ax[i,0].set_ylabel('depth [km]')
    ax[i,0].set_ylim((-5.5,0))
    for j in range(3):
        if j in [0, 2]:
            ax[i,j].set_xlim(-55,20)
            ax[i,j].set_xticks(np.arange(-40, 30, 20))
            ax[i,j].set_xticklabels(np.arange(-40, 30, 20))

ax[0,0].set_title(r'$S(y,z)$')
im0 = ax[0,0].pcolormesh(X, Y, ds.SALT, vmin=34, vmax=35.5, cmap='cmo.haline_r')
plt.colorbar(im0, ax=ax[0,0], orientation='horizontal')

ax[0,1].plot(ds.SALT.mean('nlon', skipna=True), -ds.z_t/1e5)
ax[0,1].axvline(S0, c='C1', label=f'{S0_str} = {S0.values:6.3f} g/kg')
ax[0,1].legend(frameon=False, handlelength=.8)
ax[0,1].set_title(r'$\langle S \rangle (z)$ and $S_0$')
ax[0,1].set_xlabel('salinity [g/kg]')

ax[0,2].set_title(r'$S(y,z) - \langle S \rangle (z)$')
im2 = ax[0,2].pcolormesh(X, Y, ds.SALT-ds.SALT.mean('nlon'), vmin=-.5, vmax=.5, cmap='RdBu_r')
plt.colorbar(im2, ax=ax[0,2], orientation='horizontal')


ax[1,0].set_title(r'$VVEL(y,z)$')
im0 = ax[1,0].pcolormesh(X, Y, ds.VVEL, vmin=-20, vmax=20, cmap='BrBG')
plt.colorbar(im0, ax=ax[1,0], orientation='horizontal')

ax[1,1].plot(ds.VVEL.mean('nlon', skipna=True), -ds.z_t/1e5)
ax[1,1].axvline(v0, c='C1', label=f'{v0_str} = {v0.values:5.3f} cm/s')
ax[1,1].legend(frameon=False, handlelength=.8)
ax[1,1].set_title(r'$\langle VVEL \rangle (z)$ and $VVEL_0$')
ax[1,1].set_xlabel('salinity [g/kg]')

ax[1,2].set_title(r'$VVEL(y,z) - \langle VVEL \rangle (z)$')
im2 = ax[1,2].pcolormesh(X, Y, ds.VVEL-ds.VVEL.mean('nlon'), vmin=-10, vmax=10, cmap='RdBu_r')
plt.colorbar(im2, ax=ax[1,2], orientation='horizontal')

In [ ]:
def calc_Mov(ds, S0):
    # function given by de Vries et al. (2005)
    # can use .mean function because at 34S the grid is rectilinear
    Mov = ( -1/S0*(( ds.VVEL*ds.DXU*(ds.SALT.mean('nlon', skipna=True)-S0) )*ds.dz).sum(['nlon', 'z_t'])  )/ 1e12  # 1e12 cm^3/s = 1 Sv
    return Mov

In [ ]:
# 56:16 for ctrl monthly
# 18:43 for rcp
warnings.simplefilter("ignore")
G = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)[['DXT', 'DXU', 'z_t', 'dz']].isel({'nlat':j_34, 'nlon':slice(i_SA,i_CGH)})
for i, run in enumerate(['ctrl', 'rcp', 'lpd', 'lr1', 'lr2', 'ld', 'hq']):
    fn_out = f'{path_prace}/Mov/Mov_monthly_{run}.nc'
    try:
        assert os.path.exists(fn_out)
        if run=='ctrl':  Mov_ctrl = xr.open_dataarray(fn_out, decode_times=False)
        if run=='rcp' :  Mov_rcp  = xr.open_dataarray(fn_out, decode_times=False)
        if run=='lpd' :  Mov_lpd  = xr.open_dataarray(fn_out, decode_times=False)
        if run=='lr1' :  Mov_lr1  = xr.open_dataarray(fn_out, decode_times=False)
        if run=='lr2' :  Mov_lr2  = xr.open_dataarray(fn_out, decode_times=False)
        if run=='ld'  :  Mov_ld   = xr.open_dataarray(fn_out, decode_times=False)
    except:
        if run in ['ctrl', 'rcp', 'hq']:
            roll, j_34, i_SA, i_CGH = 0, 810, 555, 1299
        if run in ['lpd', 'lr1', 'lr2', 'ld']:
            roll, j_34, i_SA, i_CGH = 100, 84, 88, 152
        for i, (y,m,fn) in tqdm_notebook(enumerate(IterateOutputCESM(run=run, domain='ocn', tavg='monthly'))):
            ds = xr.open_dataset(fn, decode_times=False)[['VVEL', 'SALT', 'DXT', 'DXU', 'z_t', 'dz']].roll(shifts={'nlon':roll}, roll_coords=True).isel({'nlat':j_34, 'nlon':slice(i_SA,i_CGH)})
            S0 = (ds.SALT*ds.DXT*ds.dz).sum(dim=['nlon','z_t'])/(ds.DXT.where(ds.SALT>0)*ds.dz).sum(dim=['nlon','z_t'])
            Mov_ = calc_Mov(ds, S0)
            if i==0:  Mov = Mov_
            else:     Mov = xr.concat([Mov, Mov_], dim='time')
        Mov.to_netcdf(fn_out)

In [ ]:
print('aaa')

In [ ]:
print('aaa')

In [ ]:
print('aaa')

In [ ]:
(Mov_ctrl.time[1:]-Mov_ctrl.time[:-1]).plot()

In [ ]:
Mov.to_netcdf(f'{path_prace}/Mov/Mov_monthly_{run}.nc')

In [ ]:
# # creating dataset of section
# warnings.simplefilter("ignore")
# G = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False)[['DXT', 'DXU', 'z_t', 'dz']].isel({'nlat':j_34, 'nlon':slice(i_SA,i_CGH)})
# for i, run in enumerate(['ctrl', 'rcp']):
#     for i, (y,m,fn) in tqdm_notebook(enumerate(IterateOutputCESM(run=run, domain='ocn', tavg='monthly'))):
#         if i==0:  ds = xr.open_dataset(fn, decode_times=False)[['VVEL', 'SALT', 'DXT', 'DXU', 'z_t', 'dz']].isel({'nlat':j_34, 'nlon':slice(i_SA,i_CGH)})
#         else:
#             ds_ = xr.open_dataset(fn, decode_times=False)[['VVEL', 'SALT']].isel({'nlat':j_34, 'nlon':slice(i_SA,i_CGH)})
#             ds = xr.merge([ds, ds_], compat='override')
#     ds.to_netcdf(f'{path_prace}/Mov/section_monthly_VVEL_SALT_{run}.nc')

In [ ]:
f, ax = plt.subplots(2,1,figsize=(8,8), constrained_layout=True)
ax[0].set_xlim(-5,305)
ax[0].plot(Mov_ctrl.time/365, Mov_ctrl, lw=.5, alpha=.3)
ax[0].plot(Mov_ctrl.time[30:-30]/365, lowpass(Mov_ctrl,12*5)[30:-30],
           c='C0', label='CTRL 5 year lowpass filtered')
ax[0].plot(Mov_rcp.time/365 -1800, Mov_rcp, lw=.5, alpha=.3)
ax[0].plot(Mov_rcp.time[30:-30]/365-1800, lowpass(Mov_rcp,12*5)[30:-30],
           c='C1', label='RCP 5 year lowpass filtered')
ax[0].legend()
ax[0].axhline(c='grey', lw=.5)

for i in range(2):  ax[i].set_ylabel(r'$M_{ov}$ [Sv]')

for i, Mov in enumerate([Mov_lpd, Mov_lr1, Mov_lr2, Mov_ld]):
    ax[1].plot(Mov.time/365-[0,1500,1400,1500][i], Mov, c=f'C{i}', lw=.5, alpha=.3)
    ax[1].plot(Mov.time[30:-30]/365-[0,1500,1400,1500][i], lowpass(Mov,12*5)[30:-30],
               c=f'C{i}', label=['LPD', 'RCP1', 'RCP2', '2xC2'][i])
ax[1].legend()
ax[1].set_xlabel('time [model years]')
plt.tight_layout()
plt.savefig(f'{path_results}/Mov/Mov_overview')

### RCP

In [ ]:
section_rcp = xr.open_dataset(f'{path_prace}/Mov/section_monthly_VVEL_SALT_rcp.nc', decode_times=False)

In [ ]:
section_rcp

In [ ]:
from xr_regression import xr_linear_trends_2D

In [ ]:
(section_rcp.SALT.isel(time=slice(-30*12, -1)).mean(dim='time', skipna=True) \
 - section_rcp.SALT.isel(time=slice(0,30*12)).mean(dim='time', skipna=True)).plot()

In [ ]:
section_rcp.SALT.isel(time=-1).plot()

## Salinity bias
_Mecking et al. (2017)_ point out that the CMIP5 $M_{ov}$ values are influenced by salinity biases, so it is of interest to quantify these in the CESM simulations.